In [1]:
import numpy as np
np.random.seed(123)
import pandas as pd
import xgboost as xgb
import gc
from datetime import datetime
from scipy.optimize import minimize
from sklearn.metrics import mean_absolute_error


In [2]:
train = pd.read_csv("./input/train.csv")
xgb1 = pd.read_csv("./xgboost/oob_xgb_fairobj_1124.252199_2016-12-09-22-05.csv")
xgb2 = pd.read_csv("./xgboost/oob_power3_xgb_fairobj_1125.110344_2016-12-08-22-18.csv")
xgb3 = pd.read_csv("./xgboost/oob_xgb_fairobj_1124.738622_2016-12-08-03-06.csv")
xgb4 = pd.read_csv("./xgboost/oob_xgb_fairobj_1125.099072_2016-12-07-21-03.csv")
xgb5 = pd.read_csv("./xgboost/oob_power3_xgb_fairobj_1124.879544_2016-12-08-07-07.csv")
xgb6 = pd.read_csv("./xgboost/oob_xgb_fairobj_1124.456032_2016-12-08-14-34.csv")

keras1 = pd.read_csv("./keras/preds_oob1130.58667325_from fourm.csv").sort_values('id')
keras2 = pd.read_csv("./keras/preds_oob1130.400_different seed.csv").sort_values('id')
keras3 = pd.read_csv("./keras/preds_oob1131.39946641_seed400-250-50_no_early_stop.csv").sort_values('id')
keras4 = pd.read_csv("./keras/preds_oob1132.36796706_no_log_400_200_50.csv").sort_values('id')

second_level1 = pd.read_csv("./second_level_models/1117.90490574preds_oob.csv")
second_level2 = pd.read_csv("./second_level_models/1117.88037992preds_oob.csv")

In [3]:
def mae_func(weights):
    ''' scipy minimize will pass the weights as a numpy array '''
    final_prediction = 0
    for weight, prediction in zip(weights, predictions):
            final_prediction += weight*prediction

    return mean_absolute_error(Y_values, final_prediction)

def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print(' Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [4]:
Y_values = train['loss'].values
predictions = []
lls = []
wghts = []

In [5]:
predictions.append(np.array(xgb1.loss))
#predictions.append(np.array(xgb2.loss))
#predictions.append(np.array(xgb3.loss))
#predictions.append(np.array(xgb4.loss))
predictions.append(np.array(xgb5.loss))
predictions.append(np.array(xgb6.loss))

predictions.append(np.array(keras1.loss))
predictions.append(np.array(keras2.loss))
predictions.append(np.array(keras3.loss))
predictions.append(np.array(keras4.loss))
#predictions.append(np.array(keras5.loss))
#predictions.append(np.array(keras6.loss))

#predictions.append(np.array(ef1.loss))
predictions.append(np.array(second_level1.loss))
predictions.append(np.array(second_level2.loss))

In [ ]:
start_time = timer(None)
for i in range(500):
    starting_values = np.random.uniform(size=len(predictions))
    cons = ({'type':'ineq','fun':lambda w: 1.2-sum(w)})
    bounds = [(0,1)]*len(predictions)

    res = minimize(mae_func, starting_values, method='L-BFGS-B',
                   bounds=bounds, options={'disp': False, 'maxiter': 10000})

    lls.append(res['fun'])
    wghts.append(res['x'])
# Uncomment the next line if you want to see the weights and scores calculated in real time
    #print('Weights: {weights}  Score: {score}'.format(weights=res['x'], score=res['fun']))

bestSC = np.min(lls)
bestWght = wghts[np.argmin(lls)]

print('\n Ensemble Score: {best_score}'.format(best_score=bestSC))
print('\n Best Weights: {weights}'.format(weights=bestWght))

timer(start_time)

In [53]:
bestSC = np.min(lls)
bestWght = wghts[np.argmin(lls)]
print bestWght

[ 0.05753586  0.          0.          0.01344082  0.          0.01745303
  0.06075416  0.42314339  0.4311688 ]


In [55]:
model1 = pd.read_csv("./xgboost/submission_5fold-average-xgb_fairobj_1124.252199_2016-12-02-11-19.csv")
model2 = pd.read_csv("./xgboost/test_power3_xgb_fairobj_1124.879544_2016-12-08-07-07.csv")
model3 = pd.read_csv("./xgboost/test_xgb_fairobj_1124.456032_2016-12-08-14-34.csv")
model4 = pd.read_csv("./keras/submission_keras_shift_perm1130.58667325.csv").sort_values('id')
model5 = pd.read_csv("./keras/submission_keras_shift_perm1130.400.csv").sort_values('id')
model6 = pd.read_csv("./keras/submission_keras_shift_perm1131.39946641.csv").sort_values('id')
model7 = pd.read_csv("./keras/submission_keras_shift_perm1132.36796706.csv").sort_values('id')
model8 = pd.read_csv("1117.90490574_submission_keras_shift_perm.csv")
model9 = pd.read_csv("1117.90490574_submission_keras_shift_perm.csv")

In [56]:
combined = bestWght[0] * np.array(model1.loss) + \
bestWght[1] * np.array(model2.loss) +\
bestWght[2] * np.array(model3.loss) +\
bestWght[3] * np.array(model4.loss) +\
bestWght[4] * np.array(model5.loss) +\
bestWght[5] * np.array(model6.loss) +\
bestWght[6] * np.array(model7.loss) +\
bestWght[7] * np.array(model8.loss) +\
bestWght[8] * np.array(model9.loss)


In [57]:
ids = pd.read_csv('./data_prep/input/test.csv')['id']
df = pd.DataFrame({'id': ids, 'loss': combined})


In [58]:
df.to_csv('allstate'+str(bestSC) + '.csv', index = False)